# Preprocesamiento

In [115]:
import os
from nltk.stem import SnowballStemmer 
import nltk
import string
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial.distance import jaccard
from sklearn.metrics.pairwise import cosine_similarity


In [116]:
#Path de los datos 
#data_path = r'C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\training_txt'
#data_path = r'D:\U\7. Septimo\RI\ProyectoRI\data\training_txt'
data_path = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\training_txt'

In [117]:
documents = [] #  Vector de documentos
for filename in os.listdir(data_path):
    if filename.endswith('.txt'): 
        path = os.path.join(data_path, filename) #Abrimos cada archivo 
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read() # leemos
            documents.append((filename, content))

In [118]:
#Path de las stopwords
#stopwords_path = r"C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\stopwords.txt"
#stopwords_path = r"D:\U\7. Septimo\RI\ProyectoRI\data\stopwords.txt"
stopwords_path = r"C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\stopwords.txt"

In [119]:
#Abrimos el archivo 
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines()) # leemos las stopwords

In [120]:
#Stemmer se usa para reducir las palabras a su raíz
stemmer = SnowballStemmer('english')

In [121]:
#Definimos una función que normaliza el texto con todos los requisitos necesarios:
def preprocess_text(text):
    text = text.lower() #conviertimos en minúsculas
    text = re.sub(r'\d+', '', text)  
    text = text.translate(str.maketrans('', '', string.punctuation))#eliminamos los signos de puntuación
    tokens = nltk.word_tokenize(text)#tokenizamos
    processed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words] #aplicamos stemming
    return ' '.join(processed_tokens)

In [122]:
preprocessed_documents = [(filename, preprocess_text(content)) for filename, content in documents]

# Cargar data importante

In [123]:
# Leer consultas
query_path = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\querys_limpio.txt'

In [124]:
#Abrimos el archivo 
with open(query_path, 'r', encoding='utf-8') as file:
    querys_of_path = set(file.read().splitlines()) # leemos las querys

In [189]:
print(querys_of_path)

{'rand income palmkernel gas ship dmk jet cotton-oil coconut housing dlr soy-oil lumber nzdlr livestock crude copra-cake hog wpi lei naphtha palladium propane rye jobs cpi gold cocoa lin-oil oat carcass bop trade rubber groundnut-oil veg-oil rapeseed sun-meal interest gnp cotton money-fx orange earn l-cattle yen rice sorghum oilseed cpu soy-meal groundnut rape-oil money-supply sunseed platinum nkr wheat reserves coconut-oil coffee iron-steel zinc potato acq silver nat-gas alum nickel soybean dfl lead meal-feed retail instal-debt pet-chem tea copper ipi corn palm-oil tin strategic-metal barley fuel heat castor-oil sugar grain sun-oil'}


# Vectorizacion

In [125]:
# Extraer solo los contenidos preprocesados para la vectorización
preprocessed_contents = [content for _, content in preprocessed_documents]

### BoW 

#### Jaccard

In [199]:
# Inicializar el vectorizador
vectorizer_bow = CountVectorizer(binary=True)

# Vectorizar los contenidos preprocesados
X_bow = vectorizer_bow.fit_transform(preprocessed_contents).toarray()

In [ ]:
# Vectorizar consultas
querys_of_path_dict = {query: 1 for query in querys_of_path}  # Asumiendo que cada consulta tiene un valor de 1
X_querys_BoW = vectorizer_bow.transform(list(querys_of_path_dict.keys())).toarray()

In [217]:
X_bow_array = X_bow.toarray()
X_querys_BoW_array = X_querys_BoW.toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [206]:
def calculate_jaccard_similarities(query_vector, document_vectors):
    similarities = []
    for doc_vector in document_vectors:
        similarity = 1 - jaccard(query_vector, doc_vector)
        similarities.append(similarity)
    return similarities

In [216]:
# Calcular y mostrar similitudes de Jaccard para cada consulta
for i, query_vector in enumerate(X_querys_BoW):
    similarities = calculate_jaccard_similarities(query_vector, X_bow_array)
    print(f"Similitudes de Jaccard para la consulta '{i}': {similarities}")

Similitudes de Jaccard para la consulta '0': [0.015789473684210575, 0.0, 0.010638297872340385, 0.0, 0.009803921568627416, 0.0, 0.015873015873015928, 0.0, 0.02083333333333337, 0.018018018018018056, 0.018181818181818188, 0.0, 0.0, 0.01041666666666663, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03508771929824561, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010869565217391353, 0.0, 0.0, 0.012345679012345734, 0.0, 0.0, 0.0, 0.0, 0.0, 0.011363636363636354, 0.008403361344537785, 0.025000000000000022, 0.0, 0.0, 0.0, 0.01941747572815533, 0.012987012987012991, 0.01041666666666663, 0.019011406844106515, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022727272727272707, 0.020725388601036232, 0.0, 0.0, 0.007575757575757569, 0.010752688172043001, 0.0, 0.0, 0.0, 0.0, 0.011111111111111072, 0.0066225165562914245, 0.0, 0.008849557522123908, 0.02577319587628868, 0.018181818181818188, 0.0, 0.0, 0.0, 0.00877192982456143, 0.0, 0.010309278350515427, 0.0, 0.0, 0.023809523809523836, 0.00666666666666671, 0.00454545454545451

### TF IDF

#### Coseno

In [130]:
# Inicializar el vectorizador
vectorizer_tfIdf = TfidfVectorizer(binary=True)
# Vectorizar los contenidos preprocesados
X_tfIdf = vectorizer_tfIdf.fit_transform(preprocessed_contents)
#Vectorizar consultas
X_querys_tfIdf = vectorizer_tfIdf.transform(querys_of_path)

In [131]:
X_tfIdf_array = X_bow.toarray()
X_querys_tfIdf_array = X_querys_tfIdf.toarray()

In [132]:
def calculate_cosine_similarities(query_vector, document_vectors):
    similarities = cosine_similarity(query_vector, document_vectors)
    return similarities.flatten()

In [140]:
for i, query_vector in enumerate(X_querys_tfIdf_array):
    query_vector_2d = query_vector.reshape(1, -1)  # Convertir a 2D
    similarities_tdIDF = calculate_cosine_similarities(query_vector_2d, X_tfIdf_array)

# Motor de busqueda

In [134]:
def leer_documentos_relevantes(archivo):
    documentos_relevantes = {}
    with open(archivo, 'r') as file:
        for line in file:
            partes = line.strip().split()
            if len(partes) != 4:  # Asegurarse de que hay 4 partes en cada línea
                continue  # Ignorar líneas que no siguen el formato esperado
            numero_documento = partes[1][:-4]  # Obtener el número de documento eliminando la extensión .txt
            try:
                similitud = float(partes[-1])  # Convertir la similitud a un número decimal
            except ValueError:
                continue  # Ignorar líneas donde la similitud no es un número válido
            documentos_relevantes[numero_documento] = similitud
    return documentos_relevantes


In [135]:
# Leer documentos relevantes
documentos_relevantes = leer_documentos_relevantes(r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\catslimpia.txt')
#documentos_relevantes = leer_documentos_relevantes('C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\catslimpia.txt.txt')


In [136]:
def calcular_precision_recall(resultados_recuperados, resultados_relevantes):
    num_resultados_recuperados = len(resultados_recuperados)
    num_resultados_relevantes = len(resultados_relevantes)
    
    resultados_comunes = set(resultados_recuperados) & set(resultados_relevantes)
    num_resultados_comunes = len(resultados_comunes)
    
    precision = num_resultados_comunes / num_resultados_recuperados if num_resultados_recuperados > 0 else 0
    recall = num_resultados_comunes / num_resultados_relevantes if num_resultados_relevantes > 0 else 0
    
    return precision, recall

In [137]:
resultados_totales = {}  # Almacenar los resultados de todas las consultas
for consulta in consultas:
    resultados_recuperados = engine.search(consulta)
    resultados_totales[consulta] = resultados_recuperados

KeyboardInterrupt: 

In [ ]:
# Calcular precisión y recall para cada consulta
precisiones = []
recalls = []
for consulta, documentos_recuperados in resultados_totales.items():
    precision, recall = calcular_precision_recall(documentos_recuperados, documentos_relevantes)
    precisiones.append(precision)
    recalls.append(recall)

# Metricas de Evaluacion

In [ ]:
# Calcular la precisión y el recall promedio para todas las consultas
precision_promedio = sum(precisiones) / len(precisiones)
recall_promedio = sum(recalls) / len(recalls)

print("Precisión promedio:", precision_promedio)
print("Recall promedio:", recall_promedio)

Precisión promedio: 0.0
Recall promedio: 0.0
